In [50]:
import boto3
from boto3.dynamodb.conditions import Attr
from datetime import datetime
import requests

In [45]:
#conectando ao dynamo
dynamodb = boto3.resource('dynamodb', 
                          region_name='us-east-1', 
                          aws_access_key_id='#######', 
                          aws_secret_access_key='######')

In [46]:
#fazendo a query para pegar os cpfs consultados no dia de hoje na boa vista (que está com o nome de serasa)
tabelaBuscaCPFs = dynamodb.Table('applicant-bureau-api-data-prod')
f = Attr('bureau_client_solicitation_ids').contains('serasa')
response = tabelaBuscaCPFs.scan(FilterExpression=f)

In [47]:
#dados que vamos utilizar 
login = "18641"
senha = "FC#2005"
produto = "690"
tabelaSalvarResultados = dynamodb.Table('boa-vista-690')
cpfsAnalisadosHoje = []

In [48]:
#percorrendo os resultados
while 'LastEvaluatedKey' in response:
    for i in response['Items']:
        dataAnaliseCPF = datetime.strptime(i['dated'], '%Y-%m-%d %H:%M:%S').date()
        if dataAnaliseCPF == date.today():
            cpfsAnalisadosHoje.append(i['document'])
            print('CPF analisado hoje: ', i['document'])
    if len(cpfsAnalisadosHoje) >= 5:
        break;
    elif 'LastEvaluatedKey' in response:
        response = table.scan(FilterExpression=f, ExclusiveStartKey=response['LastEvaluatedKey'])
        
cpfsAnalisadosHoje

CPF analisado hoje:  11646480716
CPF analisado hoje:  51548976920
CPF analisado hoje:  38455935987
CPF analisado hoje:  10831510706
CPF analisado hoje:  75812746753


['11646480716', '51548976920', '38455935987', '10831510706', '75812746753']

In [57]:
#fazendo consulta ao boa vista e gravando no dynamo
url_api_boa_vista = "https://www.scpc.inf.br/cgi-bin/spcaxml"
header = { 'Accept': 'application/text' }

for cpf in cpfsAnalisadosHoje:
    print('Consultando CPF ', cpf)
    body = "<SPCA-XML xmlns:xsi='http://www.w3.org/2001/XMLSchema-instance' xsi:noNamespaceSchemaLocation='https://www.scpc.inf.br/spcn/spcaxmldef.xsd'> <VERSAO>20151030</VERSAO> <SOLICITACAO> <S-CODIGO>"+login+"</S-CODIGO> <S-SENHA>"+senha+"</S-SENHA> <S-CONSULTA>"+produto+"</S-CONSULTA> <S-CPF>"+cpf+"</S-CPF> <S-TIPO-CREDITO>63</S-TIPO-CREDITO> </SOLICITACAO> </SPCA-XML>"
    responseBoaVista = requests.get(url_api_boa_vista, headers=header, data=body)
    tabelaSalvarResultados.put_item(
       Item={
            'cpf': cpf,
            'data': responseBoaVista.text
            }
    )
    print('Resultado: ')
    print(responseBoaVista.text)
    print('------------------------------')

Consultando CPF  11646480716
Resultado: 
<?xml version="1.0" encoding="ISO-8859-1"?>
<SPCA-XML xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="https://www.scpc.inf.br/spcn/spcaxml.xsd">
    <VERSAO>20151030</VERSAO>
    <SOLICITACAO>
        <S-CODIGO>18641</S-CODIGO>
        <S-ASSOCIADO/>
        <S-CONSULTA>690</S-CONSULTA>
        <S-CPF>11646480716</S-CPF>
        <S-TIPO-CREDITO>63</S-TIPO-CREDITO>
        <S-SCORE>63</S-SCORE>
    </SOLICITACAO>
    <RESPOSTA>
        <RESPOSTA-RETORNO>
            <STATUS-RESPOSTA>0</STATUS-RESPOSTA>
            <MENSAGEM-RESPOSTA>CONSULTA CONCLUIDA</MENSAGEM-RESPOSTA>
        </RESPOSTA-RETORNO>
        <NUMERO-RESPOSTA>286581637-4</NUMERO-RESPOSTA>
        <ASSOCIADO-SOLICITANTE>FICHA CERTA</ASSOCIADO-SOLICITANTE>
        <NOME-CONSULTA>Score Crédito PF</NOME-CONSULTA>
        <IP-SOLICITANTE>181.213.51.238</IP-SOLICITANTE>
        <REGISTRO-ACSP-SCORE>
            <SCORE-259-DADOS>
                <SCORE-